In [1]:
# --- Notebook setup ---
from pathlib import Path
import warnings, json, numpy as np
PROJECT_ROOT = Path(".").resolve()  # or set to your absolute MyModel path

# Small memory safeguards for notebooks
TOPK_INDIVIDUAL_FEATURES = 20       # down from 50
BLOCK_ABLATION_CHUNKS = 0           # disable coarse blocks
THRESHOLD = 0.5
USE_DATASET = "CTOD"

# Correct folder names
MODEL_ROOT = PROJECT_ROOT / "xgb_model_package"
DATA_ROOTS = {
    "CTOD": PROJECT_ROOT / "ctod_xgb_artifacts",
    "HINT": PROJECT_ROOT / "hint_xgb_artifacts",
}

# Optional: show where we are and what exists
print("CWD:", PROJECT_ROOT)
print("Models dir exists:", MODEL_ROOT.exists())
print("CTOD dir exists:", DATA_ROOTS["CTOD"].exists())


CWD: C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)
Models dir exists: False
CTOD dir exists: False


In [3]:
# === Ablation Study Organizer === 
# Purpose: standardize folders and output paths inside MyModel/

from __future__ import annotations
import sys, json, shutil, platform
from datetime import datetime
from pathlib import Path

# 1) Find project root (folder that contains 'xgb_model_package' OR 'ctod_xgb_artifacts')
def find_project_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "xgb_model_package").exists() or (cur / "ctod_xgb_artifacts").exists():
            return cur
        cur = cur.parent
    return start.resolve()

CWD = Path.cwd()
PROJECT_ROOT = find_project_root(CWD)

# 2) Create target structure
ABLATIONS_DIR = PROJECT_ROOT / "ablations"
NOTEBOOKS_DIR = ABLATIONS_DIR / "notebooks"
RESULTS_DIR = ABLATIONS_DIR / "results"
CSV_DIR = RESULTS_DIR / "csv"
EXCEL_DIR = RESULTS_DIR / "excel"
PLOTS_DIR = RESULTS_DIR / "plots"
LOGS_DIR = ABLATIONS_DIR / "logs"

for d in [ABLATIONS_DIR, NOTEBOOKS_DIR, RESULTS_DIR, CSV_DIR, EXCEL_DIR, PLOTS_DIR, LOGS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"[OK] Project root: {PROJECT_ROOT}")
print(f"[OK] Created/verified: {ABLATIONS_DIR}")

# 3) Helpers you’ll use later when saving outputs
def csv_target(dataset: str, phase: str, topk: int|None=None) -> Path:
    name = f"{dataset.lower()}_phase_{phase}"
    if topk is not None:
        name += f"_topk{topk}"
    return CSV_DIR / f"{name}.csv"

def plot_target(dataset: str, phase: str, top: int=10) -> Path:
    return PLOTS_DIR / f"{dataset.lower()}_phase_{phase}_top{top}.png"

def excel_target() -> Path:
    return EXCEL_DIR / "ablation_summary.xlsx"

print("[OK] Helpers available: csv_target(), plot_target(), excel_target()")

# 4) (Optional) migrate legacy outputs if they exist
legacy_dirs = [
    PROJECT_ROOT / "ablation_results",
    PROJECT_ROOT / "ablations" / "ablation_results",
]

migrated = []
for legacy in legacy_dirs:
    if legacy.exists():
        for p in legacy.rglob("*.csv"):
            dest = CSV_DIR / p.name
            if not dest.exists():
                shutil.copy2(p, dest); migrated.append(("CSV", str(p), str(dest)))
        for p in legacy.rglob("*.xlsx"):
            dest = EXCEL_DIR / p.name
            if not dest.exists():
                shutil.copy2(p, dest); migrated.append(("EXCEL", str(p), str(dest)))
        for p in legacy.rglob("*.png"):
            dest = PLOTS_DIR / p.name
            if not dest.exists():
                shutil.copy2(p, dest); migrated.append(("PLOT", str(p), str(dest)))

if migrated:
    print("[OK] Migrated legacy results:")
    for kind, src, dst in migrated:
        print(f"   - {kind}: {src} -> {dst}")
else:
    print("[OK] No legacy results to migrate (or already migrated).")

# 5) Write README_ablation.md if missing
readme_path = PROJECT_ROOT / "README_ablation.md"
if not readme_path.exists():
    readme = """# Ablation Study — Organization & Execution

This project uses a single notebook (`ablations/notebooks/Ablation.ipynb`) to run ablations for phases I/II/III on datasets (CTOD/HINT). Outputs are saved into `ablations/results/`:

- `results/csv/`     per-phase CSVs (e.g., `ctod_phase_I_topk20.csv`)
- `results/plots/`   top-10 bar charts per phase (`ctod_phase_I_top10.png`, etc.)
- `results/excel/`   merged workbook (`ablation_summary.xlsx`)
- `logs/`            run metadata (timestamp, versions)

**How to run**
1) Open `ablations/notebooks/Ablation.ipynb`.
2) Set `USE_DATASET` and `PHASES`.
3) Run all cells. See `ablations/results/` for outputs.

**Notes**
- Inference-time ablation (zeroing columns), no retraining.
- Baselines are logged per phase.
- If feature-name JSON exists, CSVs include a `feature_name` column.
"""
    readme_path.write_text(readme, encoding="utf-8")
    print(f"[OK] README created: {readme_path}")
else:
    print(f"[OK] README already exists: {readme_path}")

# 6) Minimal run log (versions + baseline if detectable)
def collect_versions():
    vers = {"python": sys.version.split()[0], "platform": platform.platform()}
    try:
        import numpy as np; vers["numpy"] = np.__version__
    except Exception: pass
    try:
        import pandas as pd; vers["pandas"] = pd.__version__
    except Exception: pass
    try:
        import sklearn; vers["sklearn"] = sklearn.__version__
    except Exception: pass
    try:
        import xgboost; vers["xgboost"] = xgboost.__version__
    except Exception: pass
    return vers

def read_baseline_from_any_csv():
    import pandas as pd
    for p in sorted(CSV_DIR.glob("*.csv")):
        try:
            df = pd.read_csv(p, nrows=3)
            for c in ["roc_auc","pr_auc","balanced_acc"]:
                if c in df.columns:
                    return {"baseline_hint": "see per-phase CSVs", "source_csv": p.name}
        except Exception:
            continue
    return {}

RUN_INFO = {
    "timestamp": datetime.now().strftime("%Y-%m-%dT%H-%M-%S"),
    "project_root": str(PROJECT_ROOT),
    "versions": collect_versions(),
    "baseline_detected": read_baseline_from_any_csv(),
}
for name in ["SEED", "RANDOM_STATE", "GLOBAL_SEED"]:
    if name in globals():
        RUN_INFO["seed"] = globals()[name]

log_path = LOGS_DIR / f"run_{RUN_INFO['timestamp']}.json"
log_path.write_text(json.dumps(RUN_INFO, indent=2), encoding="utf-8")
print(f"[OK] Log written: {log_path}")

print("\nDefault output destinations:")
print(f" - CSVs  → {CSV_DIR}")
print(f" - Plots → {PLOTS_DIR}")
print(f" - Excel → {EXCEL_DIR}")
print(f" - Logs  → {LOGS_DIR}")

# Usage later in the notebook:
# df.to_csv(csv_target('CTOD','I', topk=20), index=False)
# fig.savefig(plot_target('CTOD','I', top=10), bbox_inches='tight', dpi=150)
# with pd.ExcelWriter(excel_target()) as xw: df_all.to_excel(xw, index=False)


[OK] Project root: C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)
[OK] Created/verified: C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\ablations
[OK] Helpers available: csv_target(), plot_target(), excel_target()
[OK] No legacy results to migrate (or already migrated).
[OK] README created: C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\README_ablation.md
[OK] Log written: C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\ablations\logs\run_2025-12-13T15-57-13.json

Default output destinations:
 - CSVs  → C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\ablations\results\csv
 - Plots → C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\ablations\results\plots
 - Excel → C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\ablations\results\excel



## Structure 
MyModel/
├─ xgb_model_package/
├─ ctod_xgb_artifacts/
├─ hint_xgb_artifacts/
├─ ablations/
│  ├─ Ablation.ipynb
│  │  └─ Ablation.ipynb
│  ├─ results/
│  │  ├─ csv/
│  │  ├─ excel/
│  │  └─ plots/
│  └─ logs/
└─ ctod_custom.py


# Ablation Study

In [5]:
pip install xlsxwriter


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# Ablation runner — self-contained, phase I/II/III, tidy outputs
# =========================

import os, re, json, time, warnings
from pathlib import Path
from typing import List, Dict, Tuple, Iterable, Optional

import numpy as np
import pandas as pd
from scipy import sparse as sp
from sklearn.metrics import roc_auc_score, average_precision_score, balanced_accuracy_score
from sklearn.utils import check_random_state
from sklearn.base import BaseEstimator, TransformerMixin
from IPython.display import display
import joblib
import matplotlib.pyplot as plt


warnings.filterwarnings("ignore")

# -------------------------
# Basic config
# -------------------------
USE_DATASET = "CTOD"               # or "HINT"
PHASES: Iterable[str] = ["I", "II", "III"]
PRIMARY_METRIC = "roc_auc"         # (roc_auc | pr_auc | bal_acc)
TOPK_INDIVIDUAL_FEATURES = 20      # used for file naming only (now less relevant)
MAX_FEATURES: Optional[int] = None # None = all features (we will not use feature-level ablations now)

RANDOM_STATE = 42

# Artifacts (models + matrices)
ARTIF_ROOT = Path(r"C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\MyModel(hybrid)-2\xgb_model_package")
MATS_ROOT  = Path(r"C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\MyModel(hybrid)-2\hint_xgb_artifacts")   # change to hint_xgb_artifacts if needed

# Output folders (tidy)
RESULTS_DIR = Path("ablations/results")
CSV_DIR     = RESULTS_DIR / "csv"
PLOTS_DIR   = RESULTS_DIR / "plots"
EXCEL_DIR   = RESULTS_DIR / "excel"
for d in [CSV_DIR, PLOTS_DIR, EXCEL_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Helper path builders (so names are consistent)
def csv_target(dataset: str, phase: str, topk: int|None=None) -> Path:
    name = f"{dataset.lower()}_phase_{phase}"
    if topk is not None:
        name += f"_topk{topk}"
    return CSV_DIR / f"{name}.csv"

def plot_target(dataset: str, phase: str, top: int=10) -> Path:
    return PLOTS_DIR / f"{dataset.lower()}_phase_{phase}_top{top}.png"

def excel_target() -> Path:
    return EXCEL_DIR / "ablation_summary.xlsx"

# Optional group-definition json (not required)
FEATURE_GROUPS_JSON = Path("./feature_groups.json")

# -------------------------
# Shims for custom classes referenced in pickles
# -------------------------
def _ensure_class(name: str):
    if name in globals():
        return
    class _Shim(BaseEstimator, TransformerMixin):
        def __init__(self, *args, **kwargs): pass
        def fit(self, X, y=None): return self
        def transform(self, X):
            try:
                if hasattr(X, "to_numpy"):
                    arr = X.to_numpy()
                else:
                    arr = np.asarray(X)
                if arr.ndim == 1:
                    arr = arr.reshape(-1, 1)
                return arr
            except Exception:
                return X
        def get_feature_names_out(self, input_features=None):
            return np.array(input_features if input_features is not None else [])
    _Shim.__name__ = name
    globals()[name] = _Shim

for _nm in [
    "DelimitedMultiHot", "SeriesAsArray", "ICDMultiHot",
    "SMILESMultiHot", "CPTMultiHot", "ATCMultiHot", "SNOMEDMultiHot", "NDCMultiHot"
]:
    _ensure_class(_nm)

# -------------------------
# I/O helpers
# -------------------------
def _prefer_model(files: List[Path]) -> Path:
    """Prefer real XGB model; avoid preprocessors."""
    ranked = sorted(
        files,
        key=lambda p: (
            -int("xgb" in p.name.lower()),
            -int("model" in p.name.lower()),
            int("preprocessor" in p.name.lower()),
            p.name.lower()
        )
    )
    return ranked[0]

def _load_json(p: Path):
    with p.open("r", encoding="utf-8") as f:
        return json.load(f)

def _auto_pick(path_phase: Path, basename_main: str, basename_reduced: str) -> Optional[Path]:
    """Prefer reduced variant if present; otherwise main; else None."""
    cand_red  = path_phase / basename_reduced
    cand_main = path_phase / basename_main
    if cand_red.exists():  return cand_red
    if cand_main.exists(): return cand_main
    return None

def load_joined_test_artifacts(path_phase: Path):
    """
    Returns: X_test (csr_matrix), y_test (ndarray), feature_names (list[str]), meta (dict or None)
    Accepts either full or reduced variants. No mask required.
    """
    path_phase = Path(path_phase)
    Xp = _auto_pick(path_phase,
                    r"C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\MyModel(hybrid)-2\ctod_xgb_artifacts\phase_I\X_test_joined.npz",
                    r"C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\MyModel(hybrid)-2\ctod_xgb_artifacts\phase_I\X_test_joined_reduced.npz")
    yp = path_phase / r"C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\MyModel(hybrid)-2\ctod_xgb_artifacts\phase_I\y_test_joined.npy"
    fp = _auto_pick(path_phase,
                    r"C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\MyModel(hybrid)-2\ctod_xgb_artifacts\phase_I\feature_names_joined.json",
                    r"C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\MyModel(hybrid)-2\ctod_xgb_artifacts\phase_I\feature_names_joined_reduced_with_top_original.json")
    mp = path_phase / "preprocessor_meta_joined.json"

    missing = []
    if Xp is None: missing.append("X_test_joined(.npz) [full or reduced]")
    if not yp.exists(): missing.append("y_test_joined.npy")
    if fp is None: missing.append("feature_names_joined(.json) [full or reduced]")
    if missing:
        raise FileNotFoundError(f"Missing test artifacts in {path_phase}: {', '.join(missing)}")

    X_test = sp.load_npz(Xp)
    y_test = np.load(yp, allow_pickle=True)
    feature_names = _load_json(fp)
    meta = _load_json(mp) if mp.exists() else None

    if len(feature_names) != X_test.shape[1]:
        raise ValueError(
            f"{path_phase}: feature count mismatch — X has {X_test.shape[1]} cols, names list has {len(feature_names)}."
        )
    return X_test, y_test, feature_names, meta

def _phase_paths(phase: str) -> Tuple[Path, Path]:
    model_dir = ARTIF_ROOT / f"phase_{phase}"
    mats_dir  = MATS_ROOT  / f"phase_{phase}"
    return model_dir, mats_dir

def _load_phase(phase: str):
    model_dir, mats_dir = _phase_paths(phase)
    mdl_files = [p for p in model_dir.glob("*") if p.suffix in (".joblib", ".pkl")]
    assert mdl_files, f"No model file found in {model_dir}"
    model_file = _prefer_model(mdl_files)
    print(f"Loading model: {model_file.name}")
    model = joblib.load(model_file)  # shims handle custom classes
    X_test, y_test, feature_names, meta = load_joined_test_artifacts(mats_dir)
    return model, X_test, y_test, feature_names, mats_dir

# -------------------------
# Ablation helpers
# -------------------------
def _safe_set_columns_scalar(X, cols: List[int], fill: float):
    """Set selected columns to scalar 'fill' for CSR/CSC sparse or dense (no global densify)."""
    cols = np.asarray(cols, dtype=int)
    if cols.size == 0:
        return X
    if sp.isspmatrix(X):
        n_rows, _ = X.shape
        Xc = X.tocsc(copy=True)
        # zero existing values in those columns
        for j in cols:
            Xc.data[Xc.indptr[j]:Xc.indptr[j+1]] = 0.0
        Xc.eliminate_zeros()
        # assign constant column values
        const_col = sp.csr_matrix(np.full((n_rows, 1), float(fill)))
        for j in cols:
            Xc[:, j] = const_col
        return Xc.tocsr()
    else:
        Xc = X.copy()
        Xc[:, cols] = float(fill)
        return Xc

# match your naming better: text__, diseases__, drugs__, num__, cat__/onehot__
_NUMERIC_LIKE_PREFIX = re.compile(r"^(num__|structured__|cat__|bin__|str__|oh__|onehot__|svd_)", re.I)

def _guess_fill_for_indices(indices: List[int], feature_names: Optional[List[str]]):
    """Return 'median' for numeric-ish names; 0.0 for TF-IDF/one-hots."""
    if not feature_names:
        return 0.0
    looks_numeric = any(
        (i < len(feature_names)) and (_NUMERIC_LIKE_PREFIX.search(str(feature_names[i])) is not None)
        for i in indices
    )
    return "median" if looks_numeric else 0.0

def _column_median(X, j: int) -> float:
    """Median of column j without global densification."""
    if sp.isspmatrix(X):
        return float(np.median(X[:, j].toarray().ravel()))
    else:
        return float(np.median(X[:, j]))

def _predict_proba_binary(model, X):
    """Return positive-class probabilities across common APIs."""
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X)
        if isinstance(proba, np.ndarray) and proba.ndim == 2 and proba.shape[1] == 2:
            return proba[:, 1]
        if isinstance(proba, np.ndarray) and proba.ndim == 1:
            return proba
    if hasattr(model, "decision_function"):
        dec = model.decision_function(X)
        return 1.0 / (1.0 + np.exp(-dec))
    pred = model.predict(X)
    if isinstance(pred, np.ndarray) and pred.ndim == 1 and np.any((pred > 0) & (pred < 1)):
        return pred
    return pred.astype(float)

def _metrics(y_true, y_prob) -> Dict[str, float]:
    return {
        "roc_auc": roc_auc_score(y_true, y_prob),
        "pr_auc": average_precision_score(y_true, y_prob),
        "bal_acc": balanced_accuracy_score(y_true, (y_prob >= 0.5).astype(int)),
    }

# ----------------------------------------------------------
# Methodology-based grouping for CTOD
#      Text / Drugs-Chem / Diseases
# ----------------------------------------------------------
def assign_methodology_group(feat_name: str) -> Optional[str]:
    """
    Map a feature name to one of the methodology groups:
      - 'Text'
      - 'Drugs/Chem'
      - 'Diseases'
      - None (if it does not belong to these groups)

    The mapping is based on CTOD raw fields and feature transformations:
    - Text: features from description / criteria (POS, semantic, SVD, etc.)
    - Drugs/Chem: features from drugs, drug_type, smiles
    - Diseases: features from diseases, icdcodes
    """

    if not isinstance(feat_name, str):
        return None

    # --- TEXT FEATURES (description + criteria free text) ---
    text_patterns = [
        r"^pos__criteria__", r"^sem__criteria__",
        r"^pos__description__", r"^sem__description__",
        r"^tfidf__criteria__", r"^tfidf__description__",
        r"^svd__text__", r"^text_svd__", r"^svd__criteria__", r"^svd__description__",
    ]
    for pat in text_patterns:
        if re.search(pat, feat_name):
            return "Text"

    # --- DRUGS / CHEM FEATURES (drugs, drug_type, smiles) ---
    drugs_patterns = [
        r"^drugs_fusion__drugs__", r"^drugs_fusion__drug_type__",
        r"^drug_type__", r"^smiles__sem__", r"^smiles_svd__",
        r"__smiles__", r"^chem_",
    ]
    for pat in drugs_patterns:
        if re.search(pat, feat_name):
            return "Drugs/Chem"

    # --- DISEASES / ICD FEATURES (diseases, icdcodes) ---
    diseases_patterns = [
        r"^diseases_dual__", r"^diseases__",
        r"^icdcodes__", r"^icd", r"diagnosis",
    ]
    for pat in diseases_patterns:
        if re.search(pat, feat_name):
            return "Diseases"

    return None


def build_methodology_groups(feature_names: List[str]) -> Dict[str, List[int]]:
    """
    Build {group_name -> list of column indices} according to the methodology
    (Text, Drugs/Chem, Diseases). Features that do not match are ignored.
    """
    groups = {"Text": [], "Drugs/Chem": [], "Diseases": []}
    for idx, fn in enumerate(feature_names):
        grp = assign_methodology_group(str(fn))
        if grp in groups:
            groups[grp].append(idx)
    # remove empty groups
    return {k: v for k, v in groups.items() if v}

# -------------------------
# Ablation core (single phase)
# -------------------------
def run_ablation_for_phase(phase: str, rng=None) -> pd.DataFrame:
    t0 = time.time()
    rng = check_random_state(rng if rng is not None else RANDOM_STATE)

    print(f"\n=== Phase {phase} ===")
    model, X, y, feature_names, mats_dir = _load_phase(phase)
    n, d = X.shape
    print(f"Test shape: n={n:,}  d={d:,}")

    # baseline
    p_base = _predict_proba_binary(model, X)
    base = _metrics(y, p_base)
    print(f"Baseline  ROC-AUC={base['roc_auc']:.4f} | PR-AUC={base['pr_auc']:.4f} | BalAcc={base['bal_acc']:.4f}")

    # --- NEW: methodology groups (Text / Drugs-Chem / Diseases) ---
    groups = build_methodology_groups(feature_names)
    print(f"Loaded {len(groups)} methodology groups: {list(groups.keys())}")

    rows = []

    # ---- group ablations (ONLY methodology groups) ----
    for gname, idxs in groups.items():
        fill = _guess_fill_for_indices(idxs, feature_names)
        if fill == "median":
            if sp.isspmatrix(X):
                meds = np.array([_column_median(X, j) for j in idxs], dtype=float)
                Xabl = X.tocsr(copy=True)
                for val, j in zip(meds, idxs):
                    Xabl = _safe_set_columns_scalar(Xabl, [j], float(val))
            else:
                Xabl = X.copy()
                Xabl[:, idxs] = np.median(X[:, idxs], axis=0)
        else:
            Xabl = _safe_set_columns_scalar(X, idxs, 0.0)

        p = _predict_proba_binary(model, Xabl)
        m = _metrics(y, p)

        rows.append({
            "phase": phase,
            "ablation_type": "group",
            "label": gname,
            "n_columns": len(idxs),
            "roc_auc": m["roc_auc"],
            "pr_auc": m["pr_auc"],
            "bal_acc": m["bal_acc"],
            "delta_roc_auc": base["roc_auc"] - m["roc_auc"],
            "delta_pr_auc": base["pr_auc"] - m["pr_auc"],
            "delta_bal_acc": base["bal_acc"] - m["bal_acc"],
        })

    # NOTE: we intentionally skip per-feature ablations here
    # to avoid huge tables and focus only on Text / Drugs-Chem / Diseases.

    df = pd.DataFrame(rows)

    # ---- persist phase CSV
    out_csv = csv_target(USE_DATASET, phase, TOPK_INDIVIDUAL_FEATURES)
    df.to_csv(out_csv, index=False)
    print(f"[OK] Saved CSV: {out_csv}  ({df.shape[0]} rows)  in {time.time()-t0:.1f}s")

    # ---- quick heads-up on groups (no feature-level ranking now)
    harm_col = {"roc_auc":"delta_roc_auc", "pr_auc":"delta_pr_auc", "bal_acc":"delta_bal_acc"}[PRIMARY_METRIC]
    print("\nGroups ranked by performance drop (Δ down):")
    display(
        df.sort_values(harm_col, ascending=False)[
            ["label","n_columns","delta_roc_auc","delta_pr_auc","delta_bal_acc"]
        ]
    )

    # ---- optional simple plot per phase (bar chart per group)
    plt.figure(figsize=(6, 4))
    plt.bar(df["label"], df["delta_roc_auc"])
    plt.xticks(rotation=0)
    plt.ylabel("ΔROC-AUC")
    plt.title(f"Phase {phase} – methodology groups")
    plt.tight_layout()
    fig_path = plot_target(USE_DATASET, phase, top=len(df))
    plt.savefig(fig_path, dpi=150, bbox_inches="tight")
    plt.close()
    print(f"[OK] Plot saved: {fig_path}")

    return df

# -------------------------
# Run all requested phases and save merged outputs
# -------------------------
all_rows = []
for ph in PHASES:
    try:
        df_ph = run_ablation_for_phase(ph, rng=RANDOM_STATE)
        all_rows.append(df_ph)
    except Exception as e:
        print(f"[WARN] Phase {ph} failed: {e}")

if all_rows:
    all_df = pd.concat(all_rows, ignore_index=True)
    all_csv = CSV_DIR / f"{USE_DATASET.lower()}_all_phases.csv"
    all_df.to_csv(all_csv, index=False)
    print(f"\n[OK] Saved all phases CSV: {all_csv} ({all_df.shape[0]} rows)")

    # pivot preview for ΔROC-AUC by group (Text / Drugs-Chem / Diseases)
    group_only = all_df[all_df["ablation_type"]=="group"].copy()
    _pivot = (
        group_only.pivot_table(index="label", columns="phase", values="delta_roc_auc", aggfunc="mean")
        .fillna(0.0)
    )
    sort_cols = sorted(list(_pivot.columns)) if _pivot.shape[1] > 1 else list(_pivot.columns)
    pivot = _pivot.sort_values(by=sort_cols, ascending=False)
    print("\nPreview — mean ΔROC-AUC per group (Text / Drugs-Chem / Diseases):")
    display(pivot)

    # Excel summary
    with pd.ExcelWriter(excel_target(), engine="xlsxwriter") as xw:
        all_df.to_excel(xw, sheet_name="groups_all_phases", index=False)
    print(f"[OK] Saved Excel summary: {excel_target()}")
else:
    print("No phases completed.")



=== Phase I ===
Loading model: hint_xgb_model.joblib
Test shape: n=5,022  d=1,000
Baseline  ROC-AUC=0.8672 | PR-AUC=0.9614 | BalAcc=0.7789
Loaded 3 methodology groups: ['Text', 'Drugs/Chem', 'Diseases']
[OK] Saved CSV: ablations\results\csv\ctod_phase_I_topk20.csv  (3 rows)  in 1.0s

Groups ranked by performance drop (Δ down):


,label,n_columns,delta_roc_auc,delta_pr_auc,delta_bal_acc
0,Text,373,0.099929,0.046985,0.070943
1,Drugs/Chem,419,0.002599,0.000605,0.006368
2,Diseases,59,0.001080,-0.000149,0.004850


[OK] Plot saved: ablations\results\plots\ctod_phase_I_top3.png

=== Phase II ===
Loading model: hint_xgb_model.joblib
Test shape: n=5,022  d=1,000
Baseline  ROC-AUC=0.8199 | PR-AUC=0.9413 | BalAcc=0.7482
Loaded 3 methodology groups: ['Text', 'Drugs/Chem', 'Diseases']
[OK] Saved CSV: ablations\results\csv\ctod_phase_II_topk20.csv  (3 rows)  in 1.1s

Groups ranked by performance drop (Δ down):


,label,n_columns,delta_roc_auc,delta_pr_auc,delta_bal_acc
0,Text,373,0.036297,0.022566,0.008790
2,Diseases,59,0.000052,0.001191,0.001777
1,Drugs/Chem,419,-0.014606,-0.010245,0.006744


[OK] Plot saved: ablations\results\plots\ctod_phase_II_top3.png

=== Phase III ===
Loading model: hint_xgb_model.joblib
Test shape: n=5,022  d=1,000
Baseline  ROC-AUC=0.7617 | PR-AUC=0.9152 | BalAcc=0.7174
Loaded 3 methodology groups: ['Text', 'Drugs/Chem', 'Diseases']
[OK] Saved CSV: ablations\results\csv\ctod_phase_III_topk20.csv  (3 rows)  in 1.4s

Groups ranked by performance drop (Δ down):


,label,n_columns,delta_roc_auc,delta_pr_auc,delta_bal_acc
2,Diseases,59,0.004424,0.000825,0.008103
1,Drugs/Chem,419,-0.019003,-0.008624,0.004283
0,Text,373,-0.030936,-0.008105,0.084728


[OK] Plot saved: ablations\results\plots\ctod_phase_III_top3.png

[OK] Saved all phases CSV: ablations\results\csv\ctod_all_phases.csv (9 rows)

Preview — mean ΔROC-AUC per group (Text / Drugs-Chem / Diseases):


phase,I,II,III
label,,,
Text,0.099929,0.036297,-0.030936
Drugs/Chem,0.002599,-0.014606,-0.019003
Diseases,0.001080,0.000052,0.004424


[OK] Saved Excel summary: ablations\results\excel\ablation_summary.xlsx


In [15]:
# Ablation runner — self-contained, phase I/II/III, tidy outputs
# =========================

import os, re, json, time, warnings
from pathlib import Path
from typing import List, Dict, Tuple, Iterable, Optional

import numpy as np
import pandas as pd
from scipy import sparse as sp
from sklearn.metrics import roc_auc_score, average_precision_score, balanced_accuracy_score
from sklearn.utils import check_random_state
from sklearn.base import BaseEstimator, TransformerMixin
from IPython.display import display
import joblib
import matplotlib.pyplot as plt


warnings.filterwarnings("ignore")

# -------------------------
# Basic config
# -------------------------
USE_DATASET = "CTOD"               # or "HINT"
PHASES: Iterable[str] = ["I", "II", "III"]
PRIMARY_METRIC = "roc_auc"         # (roc_auc | pr_auc | bal_acc)
TOPK_INDIVIDUAL_FEATURES = 20      # still used in filename, but no per-feature ablations
MAX_FEATURES: Optional[int] = None # not used now

RANDOM_STATE = 42

# Artifacts (models + matrices)
ARTIF_ROOT = Path(r"C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\MyModel(hybrid)-2\xgb_model_package")
MATS_ROOT  = Path(r"C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\MyModel(hybrid)-2\ctod_xgb_artifacts")  

# Output folders (tidy)
RESULTS_DIR = Path("ablations/results")
CSV_DIR     = RESULTS_DIR / "csv"
PLOTS_DIR   = RESULTS_DIR / "plots"
EXCEL_DIR   = RESULTS_DIR / "excel"
for d in [CSV_DIR, PLOTS_DIR, EXCEL_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Helper path builders (so names are consistent)
def csv_target(dataset: str, phase: str, topk: int|None=None) -> Path:
    name = f"{dataset.lower()}_phase_{phase}"
    if topk is not None:
        name += f"_topk{topk}"
    return CSV_DIR / f"{name}.csv"

def plot_target(dataset: str, phase: str, top: int=10) -> Path:
    return PLOTS_DIR / f"{dataset.lower()}_phase_{phase}_top{top}.png"

def excel_target() -> Path:
    return EXCEL_DIR / "ablation_summary.xlsx"

# Optional group-definition json (not required)
FEATURE_GROUPS_JSON = Path("./feature_groups.json")

# -------------------------
# Shims for custom classes referenced in pickles
# -------------------------
def _ensure_class(name: str):
    if name in globals():
        return
    class _Shim(BaseEstimator, TransformerMixin):
        def __init__(self, *args, **kwargs): pass
        def fit(self, X, y=None): return self
        def transform(self, X):
            try:
                if hasattr(X, "to_numpy"):
                    arr = X.to_numpy()
                else:
                    arr = np.asarray(X)
                if arr.ndim == 1:
                    arr = arr.reshape(-1, 1)
                return arr
            except Exception:
                return X
        def get_feature_names_out(self, input_features=None):
            return np.array(input_features if input_features is not None else [])
    _Shim.__name__ = name
    globals()[name] = _Shim

for _nm in [
    "DelimitedMultiHot", "SeriesAsArray", "ICDMultiHot",
    "SMILESMultiHot", "CPTMultiHot", "ATCMultiHot", "SNOMEDMultiHot", "NDCMultiHot"
]:
    _ensure_class(_nm)

# -------------------------
# I/O helpers
# -------------------------
def _prefer_model(files: List[Path]) -> Path:
    """Prefer real XGB model; avoid preprocessors."""
    ranked = sorted(
        files,
        key=lambda p: (
            -int("xgb" in p.name.lower()),
            -int("model" in p.name.lower()),
            int("preprocessor" in p.name.lower()),
            p.name.lower()
        )
    )
    return ranked[0]

def _load_json(p: Path):
    with p.open("r", encoding="utf-8") as f:
        return json.load(f)

def _auto_pick(path_phase: Path, basename_main: str, basename_reduced: str) -> Optional[Path]:
    """Prefer reduced variant if present; otherwise main; else None."""
    cand_red  = path_phase / basename_reduced
    cand_main = path_phase / basename_main
    if cand_red.exists():  return cand_red
    if cand_main.exists(): return cand_main
    return None

def load_joined_test_artifacts(path_phase: Path):
    """
    Returns: X_test (csr_matrix), y_test (ndarray), feature_names (list[str]), meta (dict or None)
    Accepts either full or reduced variants *within this phase folder*.
    """
    path_phase = Path(path_phase)

    Xp = _auto_pick(path_phase, "X_test_joined.npz", "X_test_joined_reduced.npz")
    yp = path_phase / "y_test_joined.npy"
    fp = _auto_pick(
        path_phase,
        "feature_names_joined.json",
        "feature_names_joined_reduced_with_top_original.json",
    )
    mp = path_phase / "preprocessor_meta_joined.json"

    missing = []
    if Xp is None:
        missing.append("X_test_joined(.npz) [full or reduced]")
    if not yp.exists():
        missing.append("y_test_joined.npy")
    if fp is None:
        missing.append("feature_names_joined(.json) [full or reduced]")
    if missing:
        raise FileNotFoundError(f"Missing test artifacts in {path_phase}: {', '.join(missing)}")

    X_test = sp.load_npz(Xp)
    y_test = np.load(yp, allow_pickle=True)
    feature_names = _load_json(fp)
    meta = _load_json(mp) if mp.exists() else None

    if len(feature_names) != X_test.shape[1]:
        raise ValueError(
            f"{path_phase}: feature count mismatch — X has {X_test.shape[1]} cols, "
            f"names list has {len(feature_names)}."
        )
    return X_test, y_test, feature_names, meta

def _phase_paths(phase: str) -> Tuple[Path, Path]:
    model_dir = ARTIF_ROOT / f"phase_{phase}"
    mats_dir  = MATS_ROOT  / f"phase_{phase}"
    return model_dir, mats_dir

def _load_phase(phase: str):
    model_dir, mats_dir = _phase_paths(phase)
    mdl_files = [p for p in model_dir.glob("*") if p.suffix in (".joblib", ".pkl")]
    assert mdl_files, f"No model file found in {model_dir}"
    model_file = _prefer_model(mdl_files)
    print(f"Loading model: {model_file.name}")
    model = joblib.load(model_file)  # shims handle custom classes
    X_test, y_test, feature_names, meta = load_joined_test_artifacts(mats_dir)
    return model, X_test, y_test, feature_names, mats_dir

# -------------------------
# Ablation helpers
# -------------------------
def _safe_set_columns_scalar(X, cols: List[int], fill: float):
    """Set selected columns to scalar 'fill' for CSR/CSC sparse or dense (no global densify)."""
    cols = np.asarray(cols, dtype=int)
    if cols.size == 0:
        return X
    if sp.isspmatrix(X):
        n_rows, _ = X.shape
        Xc = X.tocsc(copy=True)
        # zero existing values in those columns
        for j in cols:
            Xc.data[Xc.indptr[j]:Xc.indptr[j+1]] = 0.0
        Xc.eliminate_zeros()
        # assign constant column values
        const_col = sp.csr_matrix(np.full((n_rows, 1), float(fill)))
        for j in cols:
            Xc[:, j] = const_col
        return Xc.tocsr()
    else:
        Xc = X.copy()
        Xc[:, cols] = float(fill)
        return Xc

_NUMERIC_LIKE_PREFIX = re.compile(r"^(num__|structured__|cat__|bin__|str__|oh__|onehot__|svd_)", re.I)

def _guess_fill_for_indices(indices: List[int], feature_names: Optional[List[str]]):
    """Return 'median' for numeric-ish names; 0.0 for TF-IDF/one-hots."""
    if not feature_names:
        return 0.0
    looks_numeric = any(
        (i < len(feature_names)) and (_NUMERIC_LIKE_PREFIX.search(str(feature_names[i])) is not None)
        for i in indices
    )
    return "median" if looks_numeric else 0.0

def _column_median(X, j: int) -> float:
    """Median of column j without global densification."""
    if sp.isspmatrix(X):
        return float(np.median(X[:, j].toarray().ravel()))
    else:
        return float(np.median(X[:, j]))

def _predict_proba_binary(model, X):
    """Return positive-class probabilities across common APIs."""
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X)
        if isinstance(proba, np.ndarray) and proba.ndim == 2 and proba.shape[1] == 2:
            return proba[:, 1]
        if isinstance(proba, np.ndarray) and proba.ndim == 1:
            return proba
    if hasattr(model, "decision_function"):
        dec = model.decision_function(X)
        return 1.0 / (1.0 + np.exp(-dec))
    pred = model.predict(X)
    if isinstance(pred, np.ndarray) and pred.ndim == 1 and np.any((pred > 0) & (pred < 1)):
        return pred
    return pred.astype(float)

def _metrics(y_true, y_prob) -> Dict[str, float]:
    return {
        "roc_auc": roc_auc_score(y_true, y_prob),
        "pr_auc": average_precision_score(y_true, y_prob),
        "bal_acc": balanced_accuracy_score(y_true, (y_prob >= 0.5).astype(int)),
    }

# ----------------------------------------------------------
# Methodology-based grouping for CTOD
#   Subgroups + coarse groups + combinations
# ----------------------------------------------------------
def assign_methodology_subgroup(feat_name: str) -> Optional[str]:
    """
    Map a feature name to one of the methodological *subgroups*:

      - 'Text/criteria'
      - 'Text/description'
      - 'Text/other'
      - 'Drugs/names'
      - 'Drugs/classes'
      - 'Chem/SMILES'
      - 'Diseases/names'
      - 'Diseases/ICD'

    Returns None if it does not belong to these modalities.
    """

    if not isinstance(feat_name, str):
        return None

    # --- TEXT: criteria vs description vs other text ---
    if re.search(r"^pos__criteria__", feat_name) or \
       re.search(r"^sem__criteria__", feat_name) or \
       re.search(r"^tfidf__criteria__", feat_name) or \
       re.search(r"^svd__criteria__", feat_name):
        return "Text/criteria"

    if re.search(r"^pos__description__", feat_name) or \
       re.search(r"^sem__description__", feat_name) or \
       re.search(r"^tfidf__description__", feat_name) or \
       re.search(r"^svd__description__", feat_name):
        return "Text/description"

    # generic text SVD / embeddings – keep as 'other' text
    if re.search(r"^svd__text__", feat_name) or \
       re.search(r"^text_svd__", feat_name):
        return "Text/other"

    # --- DRUGS: names vs classes ---
    if re.search(r"^drugs_fusion__drugs__", feat_name):
        return "Drugs/names"

    if re.search(r"^drugs_fusion__drug_type__", feat_name) or \
       re.search(r"^drug_type__", feat_name):
        return "Drugs/classes"

    # --- CHEMISTRY: SMILES ---
    if re.search(r"^smiles__sem__", feat_name) or \
       re.search(r"^smiles_svd__", feat_name) or \
       re.search(r"__smiles__", feat_name):
        return "Chem/SMILES"

    # --- DISEASES: names vs ICD ---
    if re.search(r"^diseases_dual__", feat_name) or \
       re.search(r"^diseases__", feat_name):
        return "Diseases/names"

    if re.search(r"^icdcodes__", feat_name) or \
       re.search(r"^icd", feat_name) or \
       re.search(r"diagnosis", feat_name):
        return "Diseases/ICD"

    return None


def build_methodology_groups(feature_names: List[str]):
    """
    Returns three dicts:
      - sub_groups:  subgroup_name -> [indices]
      - coarse_groups: 'Text' | 'Drugs/Chem' | 'Diseases' -> [indices]
      - combo_groups: combined labels -> [indices]
    """
    sub_groups: Dict[str, List[int]] = {}
    for idx, fn in enumerate(feature_names):
        sub = assign_methodology_subgroup(str(fn))
        if sub is not None:
            sub_groups.setdefault(sub, []).append(idx)

    # Coarse groups: union of relevant subgroups
    coarse_groups: Dict[str, List[int]] = {"Text": [], "Drugs/Chem": [], "Diseases": []}
    for sub_label, idxs in sub_groups.items():
        prefix = sub_label.split("/")[0]
        if prefix == "Text":
            coarse_groups["Text"].extend(idxs)
        elif prefix in ("Drugs", "Chem"):
            coarse_groups["Drugs/Chem"].extend(idxs)
        elif prefix == "Diseases":
            coarse_groups["Diseases"].extend(idxs)

    # Deduplicate indices
    coarse_groups = {
        k: sorted(set(v)) for k, v in coarse_groups.items() if v
    }

    # Combinations of coarse groups
    combo_groups: Dict[str, List[int]] = {}

    def _union(label, keys):
        idxs: List[int] = []
        for k in keys:
            if k in coarse_groups:
                idxs.extend(coarse_groups[k])
        idxs = sorted(set(idxs))
        if idxs:
            combo_groups[label] = idxs

    _union("Text+Drugs/Chem", ["Text", "Drugs/Chem"])
    _union("Text+Diseases", ["Text", "Diseases"])
    _union("Drugs/Chem+Diseases", ["Drugs/Chem", "Diseases"])
    _union("All (Text+Drugs/Chem+Diseases)", ["Text", "Drugs/Chem", "Diseases"])

    # Remove empty subgroups as well
    sub_groups = {k: v for k, v in sub_groups.items() if v}

    return sub_groups, coarse_groups, combo_groups

# -------------------------
# Ablation core (single phase)
# -------------------------
def run_ablation_for_phase(phase: str, rng=None) -> pd.DataFrame:
    t0 = time.time()
    rng = check_random_state(rng if rng is not None else RANDOM_STATE)

    print(f"\n=== Phase {phase} ===")
    model, X, y, feature_names, mats_dir = _load_phase(phase)
    n, d = X.shape
    print(f"Test shape: n={n:,}  d={d:,}")

    # baseline
    p_base = _predict_proba_binary(model, X)
    base = _metrics(y, p_base)
    print(f"Baseline  ROC-AUC={base['roc_auc']:.4f} | PR-AUC={base['pr_auc']:.4f} | BalAcc={base['bal_acc']:.4f}")

    # Build subgroups, coarse groups and combinations
    sub_groups, coarse_groups, combo_groups = build_methodology_groups(feature_names)

    print("Subgroups:", list(sub_groups.keys()))
    print("Coarse groups:", list(coarse_groups.keys()))
    print("Combo groups:", list(combo_groups.keys()))

    rows = []

    # Define all ablations we want to run
    ablations: List[Tuple[str, str, List[int]]] = []  # (level, label, indices)

    # a) Sub-groups (fine-grained)
    for label, idxs in sub_groups.items():
        ablations.append(("subgroup", label, idxs))

    # a) Coarse groups (Text, Drugs/Chem, Diseases)
    for label, idxs in coarse_groups.items():
        ablations.append(("group", label, idxs))

    # b) Combinations of coarse groups
    for label, idxs in combo_groups.items():
        ablations.append(("combo", label, idxs))

    # ---- run all ablations ----
    for level, gname, idxs in ablations:
        fill = _guess_fill_for_indices(idxs, feature_names)
        if fill == "median":
            if sp.isspmatrix(X):
                meds = np.array([_column_median(X, j) for j in idxs], dtype=float)
                Xabl = X.tocsr(copy=True)
                for val, j in zip(meds, idxs):
                    Xabl = _safe_set_columns_scalar(Xabl, [j], float(val))
            else:
                Xabl = X.copy()
                Xabl[:, idxs] = np.median(X[:, idxs], axis=0)
        else:
            Xabl = _safe_set_columns_scalar(X, idxs, 0.0)

        p = _predict_proba_binary(model, Xabl)
        m = _metrics(y, p)

        # c) relative losses (Δ / baseline)
        rel_roc = (base["roc_auc"] - m["roc_auc"]) / base["roc_auc"] if base["roc_auc"] > 0 else 0.0
        rel_pr  = (base["pr_auc"] - m["pr_auc"]) / base["pr_auc"]   if base["pr_auc"] > 0 else 0.0
        rel_bal = (base["bal_acc"] - m["bal_acc"]) / base["bal_acc"] if base["bal_acc"] > 0 else 0.0

        rows.append({
            "phase": phase,
            "group_level": level,     # subgroup / group / combo
            "ablation_type": "group", # kept for compatibility
            "label": gname,
            "n_columns": len(idxs),
            "roc_auc": m["roc_auc"],
            "pr_auc": m["pr_auc"],
            "bal_acc": m["bal_acc"],
            "delta_roc_auc": base["roc_auc"] - m["roc_auc"],
            "delta_pr_auc": base["pr_auc"] - m["pr_auc"],
            "delta_bal_acc": base["bal_acc"] - m["bal_acc"],
            "rel_roc_loss": rel_roc,
            "rel_pr_loss": rel_pr,
            "rel_balacc_loss": rel_bal,
            "baseline_roc_auc": base["roc_auc"],
            "baseline_pr_auc": base["pr_auc"],
            "baseline_bal_acc": base["bal_acc"],
        })

    df = pd.DataFrame(rows)

    # ---- persist phase CSV
    out_csv = csv_target(USE_DATASET, phase, TOPK_INDIVIDUAL_FEATURES)
    df.to_csv(out_csv, index=False)
    print(f"[OK] Saved CSV: {out_csv}  ({df.shape[0]} rows)  in {time.time()-t0:.1f}s")

    # ---- quick heads-up: show coarse + combos sorted by harm ----
    harm_col = {"roc_auc":"delta_roc_auc", "pr_auc":"delta_pr_auc", "bal_acc":"delta_bal_acc"}[PRIMARY_METRIC]

    print("\nCoarse groups + combos ranked by performance drop (Δ down):")
    display(
        df[df["group_level"].isin(["group", "combo"])]
        .sort_values(harm_col, ascending=False)[
            ["phase","group_level","label","n_columns",
             "delta_roc_auc","delta_pr_auc","delta_bal_acc",
             "rel_roc_loss","rel_pr_loss","rel_balacc_loss"]
        ]
    )

    # ---- simple plot: only coarse groups (not subgroups/combos) ----
    coarse_df = df[df["group_level"] == "group"].copy()
    if not coarse_df.empty:
        plt.figure(figsize=(6, 4))
        plt.bar(coarse_df["label"], coarse_df["delta_roc_auc"])
        plt.xticks(rotation=0)
        plt.ylabel("ΔROC-AUC")
        plt.title(f"Phase {phase} – coarse groups")
        plt.tight_layout()
        fig_path = plot_target(USE_DATASET, phase, top=len(coarse_df))
        plt.savefig(fig_path, dpi=150, bbox_inches="tight")
        plt.close()
        print(f"[OK] Plot saved: {fig_path}")
    else:
        print("No coarse groups found to plot.")

    return df

# -------------------------
# Run all requested phases and save merged outputs
# -------------------------
all_rows = []
for ph in PHASES:
    try:
        df_ph = run_ablation_for_phase(ph, rng=RANDOM_STATE)
        all_rows.append(df_ph)
    except Exception as e:
        print(f"[WARN] Phase {ph} failed: {e}")

if all_rows:
    all_df = pd.concat(all_rows, ignore_index=True)
    all_csv = CSV_DIR / f"{USE_DATASET.lower()}_all_phases.csv"
    all_df.to_csv(all_csv, index=False)
    print(f"\n[OK] Saved all phases CSV: {all_csv} ({all_df.shape[0]} rows)")

    # Pivot preview for ΔROC-AUC by coarse group + combos
    group_only = all_df[all_df["group_level"].isin(["group", "combo"])].copy()
    _pivot = (
        group_only.pivot_table(index="label", columns="phase", values="delta_roc_auc", aggfunc="mean")
        .fillna(0.0)
    )
    sort_cols = sorted(list(_pivot.columns)) if _pivot.shape[1] > 1 else list(_pivot.columns)
    pivot = _pivot.sort_values(by=sort_cols, ascending=False)
    print("\nPreview — mean ΔROC-AUC per coarse/combined group (Text, Drugs/Chem, Diseases, combos):")
    display(pivot)

    # Excel summary (all rows: subgroups + groups + combos)
    with pd.ExcelWriter(excel_target(), engine="xlsxwriter") as xw:
        all_df.to_excel(xw, sheet_name="groups_all_phases", index=False)
    print(f"[OK] Saved Excel summary: {excel_target()}")
else:
    print("No phases completed.")



=== Phase I ===
Loading model: hint_xgb_model.joblib
Test shape: n=5,022  d=1,000
Baseline  ROC-AUC=0.8672 | PR-AUC=0.9614 | BalAcc=0.7789
Subgroups: ['Text/criteria', 'Drugs/names', 'Chem/SMILES', 'Diseases/names']
Coarse groups: ['Text', 'Drugs/Chem', 'Diseases']
Combo groups: ['Text+Drugs/Chem', 'Text+Diseases', 'Drugs/Chem+Diseases', 'All (Text+Drugs/Chem+Diseases)']
[OK] Saved CSV: ablations\results\csv\ctod_phase_I_topk20.csv  (11 rows)  in 2.8s

Coarse groups + combos ranked by performance drop (Δ down):


,phase,group_level,label,n_columns,delta_roc_auc,delta_pr_auc,delta_bal_acc,rel_roc_loss,rel_pr_loss,rel_balacc_loss
4,I,group,Text,373,0.099929,0.046985,0.070943,0.115236,0.048874,0.091078
8,I,combo,Text+Diseases,432,0.086901,0.046709,0.070823,0.100212,0.048586,0.090923
7,I,combo,Text+Drugs/Chem,792,0.074965,0.035458,0.063628,0.086448,0.036884,0.081686
10,I,combo,All (Text+Drugs/Chem+Diseases),851,0.070345,0.036920,0.073282,0.081121,0.038404,0.094081
9,I,combo,Drugs/Chem+Diseases,478,0.003402,-0.000497,0.010228,0.003924,-0.000517,0.013130
5,I,group,Drugs/Chem,419,0.002599,0.000605,0.006368,0.002998,0.000629,0.008175
6,I,group,Diseases,59,0.001080,-0.000149,0.004850,0.001246,-0.000155,0.006226


[OK] Plot saved: ablations\results\plots\ctod_phase_I_top3.png

=== Phase II ===
Loading model: hint_xgb_model.joblib
Test shape: n=6,738  d=1,000
Baseline  ROC-AUC=0.8330 | PR-AUC=0.9165 | BalAcc=0.7663
Subgroups: ['Text/criteria', 'Drugs/names', 'Chem/SMILES', 'Diseases/names']
Coarse groups: ['Text', 'Drugs/Chem', 'Diseases']
Combo groups: ['Text+Drugs/Chem', 'Text+Diseases', 'Drugs/Chem+Diseases', 'All (Text+Drugs/Chem+Diseases)']
[OK] Saved CSV: ablations\results\csv\ctod_phase_II_topk20.csv  (11 rows)  in 3.8s

Coarse groups + combos ranked by performance drop (Δ down):


,phase,group_level,label,n_columns,delta_roc_auc,delta_pr_auc,delta_bal_acc,rel_roc_loss,rel_pr_loss,rel_balacc_loss
4,II,group,Text,371,0.034676,0.019065,0.043668,0.041627,0.020801,0.056988
7,II,combo,Text+Drugs/Chem,769,0.032971,0.019815,0.011468,0.039580,0.021620,0.014966
8,II,combo,Text+Diseases,433,0.028782,0.015585,0.027174,0.034551,0.017005,0.035463
10,II,combo,All (Text+Drugs/Chem+Diseases),831,0.015291,0.011267,0.002587,0.018356,0.012293,0.003377
5,II,group,Drugs/Chem,398,-0.002707,0.001511,0.002478,-0.003250,0.001649,0.003234
6,II,group,Diseases,62,-0.002978,-0.001431,-0.005432,-0.003575,-0.001561,-0.007090
9,II,combo,Drugs/Chem+Diseases,460,-0.004944,-0.000252,-0.006197,-0.005935,-0.000275,-0.008087


[OK] Plot saved: ablations\results\plots\ctod_phase_II_top3.png

=== Phase III ===
Loading model: hint_xgb_model.joblib
Test shape: n=4,465  d=1,000
Baseline  ROC-AUC=0.8030 | PR-AUC=0.9378 | BalAcc=0.7302
Subgroups: ['Text/criteria', 'Drugs/names', 'Chem/SMILES', 'Diseases/names']
Coarse groups: ['Text', 'Drugs/Chem', 'Diseases']
Combo groups: ['Text+Drugs/Chem', 'Text+Diseases', 'Drugs/Chem+Diseases', 'All (Text+Drugs/Chem+Diseases)']
[OK] Saved CSV: ablations\results\csv\ctod_phase_III_topk20.csv  (11 rows)  in 3.1s

Coarse groups + combos ranked by performance drop (Δ down):


,phase,group_level,label,n_columns,delta_roc_auc,delta_pr_auc,delta_bal_acc,rel_roc_loss,rel_pr_loss,rel_balacc_loss
8,III,combo,Text+Diseases,449,0.047842,0.019901,0.122156,0.059580,0.021221,0.167286
4,III,group,Text,385,0.037332,0.014617,0.111312,0.046491,0.015587,0.152436
10,III,combo,All (Text+Drugs/Chem+Diseases),844,0.035975,0.016823,0.113823,0.044801,0.017939,0.155874
7,III,combo,Text+Drugs/Chem,780,0.030052,0.012238,0.098251,0.037425,0.013050,0.134550
6,III,group,Diseases,64,0.007362,0.003606,0.001816,0.009169,0.003845,0.002487
9,III,combo,Drugs/Chem+Diseases,459,-0.006208,-0.002385,0.029311,-0.007731,-0.002543,0.040140
5,III,group,Drugs/Chem,395,-0.009146,-0.004403,0.023809,-0.011390,-0.004696,0.032605


[OK] Plot saved: ablations\results\plots\ctod_phase_III_top3.png

[OK] Saved all phases CSV: ablations\results\csv\ctod_all_phases.csv (33 rows)

Preview — mean ΔROC-AUC per coarse/combined group (Text, Drugs/Chem, Diseases, combos):


phase,I,II,III
label,,,
Text,0.099929,0.034676,0.037332
Text+Diseases,0.086901,0.028782,0.047842
Text+Drugs/Chem,0.074965,0.032971,0.030052
All (Text+Drugs/Chem+Diseases),0.070345,0.015291,0.035975
Drugs/Chem+Diseases,0.003402,-0.004944,-0.006208
Drugs/Chem,0.002599,-0.002707,-0.009146
Diseases,0.001080,-0.002978,0.007362


[OK] Saved Excel summary: ablations\results\excel\ablation_summary.xlsx


In [17]:
from pathlib import Path
import pandas as pd
from IPython.display import display

# --------------------------------------------------
# 1) Locate the combined all-phases CSV
# --------------------------------------------------
ROOT = Path.cwd()

candidate_paths = [
    ROOT / "ablations" / "results" / "csv" / "ctod_all_phases.csv",
    ROOT / "ablations" / "results" / "csv" / "ablation_all_phases.csv",
]

combined_path = None
for p in candidate_paths:
    if p.exists():
        combined_path = p
        break

if combined_path is None:
    # Fallback: search any *all_phases*.csv
    search_root = ROOT / "ablations" / "results" / "csv"
    if search_root.exists():
        hits = list(search_root.rglob("*all_phases*.csv"))
        if hits:
            combined_path = hits[0]

if combined_path is None:
    raise FileNotFoundError("Could not find any combined all-phases CSV. "
                            "Make sure the ablation runner has been executed.")

print("[OK] Using combined CSV:", combined_path)

all_df = pd.read_csv(combined_path)
print("Shape of all_df:", all_df.shape)

# Instead of head(5), show how many rows per phase we have
print("\nNumber of rows per phase:")
print(all_df["phase"].value_counts().sort_index())

# Optional: quick preview with a few rows from each phase
print("\nPreview: first 3 rows per phase:")
for ph in sorted(all_df["phase"].unique()):
    print(f"\n--- Phase {ph} ---")
    display(all_df[all_df["phase"] == ph].head(3))


# --------------------------------------------------
# 2) Split into baseline / coarse groups / subgroups / combos
# --------------------------------------------------
baseline_df = (
    all_df[["phase", "baseline_roc_auc", "baseline_pr_auc", "baseline_bal_acc"]]
    .drop_duplicates()
    .sort_values("phase")
)

print("\nBaseline metrics per phase:")
display(baseline_df)

coarse_df   = all_df[all_df["group_level"] == "group"].copy()
subgroup_df = all_df[all_df["group_level"] == "subgroup"].copy()
combo_df    = all_df[all_df["group_level"] == "combo"].copy()

print("\nRows by group_level:")
print(all_df["group_level"].value_counts())


# --------------------------------------------------
# 3) Helper: pivot and flatten columns for Excel/report
# --------------------------------------------------
def make_pivot_table(df: pd.DataFrame, value_cols, index="label", columns="phase"):
    if isinstance(value_cols, str):
        value_cols = [value_cols]

    if df.empty:
        return pd.DataFrame()

    pivot = df.pivot_table(
        index=index,
        columns=columns,
        values=value_cols,
        aggfunc="mean"
    )

    pivot.columns = [f"{metric}_{phase}" for (metric, phase) in pivot.columns]
    pivot = pivot.reset_index()
    return pivot


# --------------------------------------------------
# 4) Build the summary tables
# --------------------------------------------------
coarse_pivot = make_pivot_table(
    coarse_df,
    value_cols=["delta_roc_auc", "rel_roc_loss", "roc_auc"]
)
print("\nCoarse groups – summary (ΔROC, relative loss, ROC):")
display(coarse_pivot)

subgroup_pivot = make_pivot_table(
    subgroup_df,
    value_cols=["delta_roc_auc", "rel_roc_loss", "roc_auc"]
)
print("\nSubgroups – summary (ΔROC, relative loss, ROC):")
display(subgroup_pivot)

combo_pivot = make_pivot_table(
    combo_df,
    value_cols=["delta_roc_auc", "rel_roc_loss", "roc_auc"]
)
print("\nCombination groups – summary (ΔROC, relative loss, ROC):")
display(combo_pivot)


# --------------------------------------------------
# 5) Export all tables to a dedicated Excel file
# --------------------------------------------------
excel_dir = ROOT / "ablations" / "results" / "excel"
excel_dir.mkdir(parents=True, exist_ok=True)

output_path = excel_dir / "ablation_report_tables_ROC_without_CI.xlsx"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    baseline_df.to_excel(writer, sheet_name="baseline_per_phase", index=False)
    coarse_pivot.to_excel(writer, sheet_name="coarse_groups", index=False)
    subgroup_pivot.to_excel(writer, sheet_name="subgroups", index=False)
    combo_pivot.to_excel(writer, sheet_name="combo_groups", index=False)

print("\n[OK] Exported report tables to:", output_path)


[OK] Using combined CSV: C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\ablations\results\csv\ctod_all_phases.csv
Shape of all_df: (33, 17)

Number of rows per phase:
phase
I      11
II     11
III    11
Name: count, dtype: int64

Preview: first 3 rows per phase:

--- Phase I ---


,phase,group_level,ablation_type,label,n_columns,roc_auc,pr_auc,bal_acc,delta_roc_auc,delta_pr_auc,delta_bal_acc,rel_roc_loss,rel_pr_loss,rel_balacc_loss,baseline_roc_auc,baseline_pr_auc,baseline_bal_acc
0,I,subgroup,group,Text/criteria,373,0.767237,0.914370,0.707986,0.099929,0.046985,0.070943,0.115236,0.048874,0.091078,0.867166,0.961355,0.77893
1,I,subgroup,group,Drugs/names,385,0.865622,0.960945,0.770632,0.001544,0.000411,0.008297,0.001780,0.000427,0.010652,0.867166,0.961355,0.77893
2,I,subgroup,group,Chem/SMILES,34,0.866281,0.960944,0.775844,0.000885,0.000412,0.003086,0.001021,0.000428,0.003962,0.867166,0.961355,0.77893



--- Phase II ---


,phase,group_level,ablation_type,label,n_columns,roc_auc,pr_auc,bal_acc,delta_roc_auc,delta_pr_auc,delta_bal_acc,rel_roc_loss,rel_pr_loss,rel_balacc_loss,baseline_roc_auc,baseline_pr_auc,baseline_bal_acc
11,II,subgroup,group,Text/criteria,371,0.798342,0.897463,0.722594,0.034676,0.019065,0.043668,0.041627,0.020801,0.056988,0.833018,0.916528,0.766262
12,II,subgroup,group,Drugs/names,366,0.836341,0.915411,0.765572,-0.003323,0.001117,0.000690,-0.003989,0.001219,0.000901,0.833018,0.916528,0.766262
13,II,subgroup,group,Chem/SMILES,32,0.833346,0.916119,0.767413,-0.000328,0.000409,-0.001151,-0.000394,0.000446,-0.001502,0.833018,0.916528,0.766262



--- Phase III ---


...


Subgroups – summary (ΔROC, relative loss, ROC):


,label,delta_roc_auc_I,delta_roc_auc_II,delta_roc_auc_III,rel_roc_loss_I,rel_roc_loss_II,rel_roc_loss_III,roc_auc_I,roc_auc_II,roc_auc_III
0,Chem/SMILES,0.000885,-0.000328,0.000653,0.001021,-0.000394,0.000813,0.866281,0.833346,0.802341
1,Diseases/names,0.001080,-0.002978,0.007362,0.001246,-0.003575,0.009169,0.866086,0.835996,0.795632
2,Drugs/names,0.001544,-0.003323,-0.008920,0.001780,-0.003989,-0.011108,0.865622,0.836341,0.811914
3,Text/criteria,0.099929,0.034676,0.037332,0.115236,0.041627,0.046491,0.767237,0.798342,0.765662



Combination groups – summary (ΔROC, relative loss, ROC):


,label,delta_roc_auc_I,delta_roc_auc_II,delta_roc_auc_III,rel_roc_loss_I,rel_roc_loss_II,rel_roc_loss_III,roc_auc_I,roc_auc_II,roc_auc_III
0,All (Text+Drugs/Chem+Diseases),0.070345,0.015291,0.035975,0.081121,0.018356,0.044801,0.796821,0.817727,0.767019
1,Drugs/Chem+Diseases,0.003402,-0.004944,-0.006208,0.003924,-0.005935,-0.007731,0.863763,0.837962,0.809202
2,Text+Diseases,0.086901,0.028782,0.047842,0.100212,0.034551,0.059580,0.780265,0.804236,0.755152
3,Text+Drugs/Chem,0.074965,0.032971,0.030052,0.086448,0.039580,0.037425,0.792201,0.800047,0.772942



[OK] Exported report tables to: C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\ablations\results\excel\ablation_report_tables_ROC_without_CI.xlsx


In [21]:
# === Bootstrap utilities for confidence intervals on Δ-metrics (coarse groups) ===
import numpy as np
import pandas as pd
from sklearn.utils import check_random_state

# _metrics and _predict_proba_binary are already defined in the previous cell
# we will reuse them here.


def _bootstrap_delta_from_proba(
    y_true: np.ndarray,
    p_base: np.ndarray,
    p_ablation: np.ndarray,
    n_bootstrap: int = 500,
    random_state: int = 42,
):
    """
    Paired bootstrap on prediction probabilities.

    For each bootstrap sample:
      - compute metrics for baseline (p_base)
      - compute metrics for ablation (p_ablation)
      - store (baseline - ablation) for each metric.

    Returns:
      dict with mean Δ and 95% CI for ROC-AUC, PR-AUC, Balanced Accuracy.
    """
    rng = check_random_state(random_state)
    n = len(y_true)

    roc_deltas = []
    pr_deltas = []
    bal_deltas = []

    for _ in range(n_bootstrap):
        idx = rng.randint(0, n, size=n)

        m_base = _metrics(y_true[idx], p_base[idx])
        m_abl  = _metrics(y_true[idx], p_ablation[idx])

        roc_deltas.append(m_base["roc_auc"] - m_abl["roc_auc"])
        pr_deltas.append(m_base["pr_auc"] - m_abl["pr_auc"])
        bal_deltas.append(m_base["bal_acc"] - m_abl["bal_acc"])

    def _summary(arr):
        arr = np.asarray(arr, dtype=float)
        mean = float(arr.mean())
        low, high = np.percentile(arr, [2.5, 97.5])
        return mean, float(low), float(high)

    roc_mean, roc_low, roc_high = _summary(roc_deltas)
    pr_mean, pr_low, pr_high    = _summary(pr_deltas)
    bal_mean, bal_low, bal_high = _summary(bal_deltas)

    return {
        "delta_roc_auc_boot": roc_mean,
        "delta_roc_auc_ci_low": roc_low,
        "delta_roc_auc_ci_high": roc_high,
        "delta_pr_auc_boot": pr_mean,
        "delta_pr_auc_ci_low": pr_low,
        "delta_pr_auc_ci_high": pr_high,
        "delta_bal_acc_boot": bal_mean,
        "delta_bal_acc_ci_low": bal_low,
        "delta_bal_acc_ci_high": bal_high,
    }


def plot_coarse_group_deltas_with_ci(df_phase: pd.DataFrame, phase_label: str):
    """
    Bar plot of ΔROC-AUC for coarse groups (Text / Drugs-Chem / Diseases)
    with 95% CI error bars, if available.

    Assumes df_phase has columns:
      - 'ablation_type' == 'group'
      - 'label'  (e.g. 'Text', 'Drugs/Chem', 'Diseases')
      - 'delta_roc_auc'
      - 'delta_roc_auc_ci_low'
      - 'delta_roc_auc_ci_high'
    """
    df_plot = df_phase[df_phase["ablation_type"] == "group"].copy()
    df_plot = df_plot.sort_values("delta_roc_auc", ascending=False)

    if df_plot.empty:
        print(f"No coarse groups to plot for phase {phase_label}.")
        return

    labels = df_plot["label"].tolist()
    y = df_plot["delta_roc_auc"].values

    has_ci = {"delta_roc_auc_ci_low", "delta_roc_auc_ci_high"}.issubset(df_plot.columns)

    if has_ci:
        lower_err = y - df_plot["delta_roc_auc_ci_low"].values
        upper_err = df_plot["delta_roc_auc_ci_high"].values - y
        yerr = np.vstack([lower_err, upper_err])
    else:
        yerr = None

    plt.figure(figsize=(6, 4))
    if yerr is not None:
        plt.bar(labels, y, yerr=yerr, capsize=4)
    else:
        plt.bar(labels, y)

    plt.axhline(0.0, linestyle="--")
    plt.ylabel("Δ ROC-AUC (baseline - ablation)")
    plt.title(f"{USE_DATASET} – Phase {phase_label} – coarse ablations with 95% CI")
    plt.xticks(rotation=0)
    plt.tight_layout()

    fig_path = PLOTS_DIR / f"{USE_DATASET.lower()}_phase_{phase_label}_coarse_ci.png"
    plt.savefig(fig_path, dpi=150, bbox_inches="tight")
    plt.close()
    print(f"[OK] Saved CI plot: {fig_path}")


In [23]:
# === Compute bootstrap CIs for ALL levels (subgroups, coarse groups, combinations)
#     and update combined CSV/Excel ===

import numpy as np
import pandas as pd
import scipy.sparse as sp

ci_rows = []

for phase in PHASES:
    print(f"\n=== Bootstrap CIs for ALL ablation levels – Phase {phase} ===")

    # Reuse your existing loading logic
    model, X, y, feature_names, mats_dir = _load_phase(phase)
    p_base = _predict_proba_binary(model, X)

    # This returns THREE dicts:
    #   sub_groups, coarse_groups, combo_groups
    sub_groups, coarse_groups, combo_groups = build_methodology_groups(feature_names)

    # Helper to avoid repeating the same code three times
    def compute_ci_for_groupdict(group_dict, level_name: str):
        """
        group_dict: dict[label -> list of feature indices]
        level_name: 'subgroup', 'group', or 'combo'
        """
        for gname, idxs in group_dict.items():
            print(f"  > [{level_name}] {gname} (n_features={len(idxs)})")

            # Use EXACTLY the same fill strategy as in run_ablation_for_phase
            fill = _guess_fill_for_indices(idxs, feature_names)

            if fill == "median":
                if sp.isspmatrix(X):
                    meds = np.array([_column_median(X, j) for j in idxs], dtype=float)
                    Xabl = X.tocsr(copy=True)
                    for val, j in zip(meds, idxs):
                        Xabl = _safe_set_columns_scalar(Xabl, [j], float(val))
                else:
                    Xabl = X.copy()
                    Xabl[:, idxs] = np.median(X[:, idxs], axis=0)
            else:
                # typically 0.0 for TF-IDF / one-hot style features
                Xabl = _safe_set_columns_scalar(X, idxs, 0.0)

            # probabilities with the ablated features
            p_abl = _predict_proba_binary(model, Xabl)

            # bootstrap paired differences (baseline - ablation) for ROC/PR/BalAcc
            boot = _bootstrap_delta_from_proba(
                y_true=y,
                p_base=p_base,
                p_ablation=p_abl,
                n_bootstrap=500,           # increase to 1000 if runtime is fine
                random_state=RANDOM_STATE,
            )

            ci_rows.append({
                "phase": phase,
                "group_level": level_name,  # matches your existing 'group_level' (group/subgroup/combo)
                "label": gname,             # e.g. 'Text/criteria', 'Text', 'Text+Drugs/Chem', etc.
                **boot,                     # all delta_*_boot and CI columns
            })

    # 1) Subgroups (e.g. Text/criteria, Drugs/names, ...)
    compute_ci_for_groupdict(sub_groups, level_name="subgroup")

    # 2) Coarse groups (Text, Drugs/Chem, Diseases)
    compute_ci_for_groupdict(coarse_groups, level_name="group")

    # 3) Combinations (Text+Drugs/Chem, All, etc.)
    compute_ci_for_groupdict(combo_groups, level_name="combo")


# Turn CI results into a DataFrame
ci_df = pd.DataFrame(ci_rows)
print("\nPreview of CI table (first rows):")
display(ci_df.head())

# Load existing all-phases CSV produced by run_ablation_for_phase
all_csv = CSV_DIR / f"{USE_DATASET.lower()}_all_phases.csv"
all_df = pd.read_csv(all_csv)

print(f"\nLoaded all-phases CSV from: {all_csv}")
print("Shape before merge:", all_df.shape)
print("Columns in all_df:", list(all_df.columns))

# Merge CI info into ALL levels via (phase, group_level, label)
#   - group_level ∈ {'group', 'subgroup', 'combo'}
#   - label: same strings as in build_methodology_groups
all_df = all_df.merge(
    ci_df,
    on=["phase", "group_level", "label"],
    how="left",
)

print("Shape after merge:", all_df.shape)

# Save updated CSV (overwrite)
all_df.to_csv(all_csv, index=False)
print(f"[OK] Updated all-phases CSV with CI columns: {all_csv}")

# Refresh Excel summary (now includes CI columns too)
with pd.ExcelWriter(excel_target(), engine="xlsxwriter") as xw:
    all_df.to_excel(xw, sheet_name="groups_all_phases", index=False)
print(f"[OK] Updated Excel summary: {excel_target()}")

# --------------------------------------------------
# Generate new CI-aware plots for each phase
# (plot only coarse groups, but we now also have CIs for subgroups and combos)
# --------------------------------------------------
for ph in PHASES:
    df_phase = all_df[all_df["phase"] == ph].copy()
    plot_coarse_group_deltas_with_ci(df_phase, phase_label=ph)



=== Bootstrap CIs for ALL ablation levels – Phase I ===
Loading model: hint_xgb_model.joblib
  > [subgroup] Text/criteria (n_features=373)
  > [subgroup] Drugs/names (n_features=385)
  > [subgroup] Chem/SMILES (n_features=34)
  > [subgroup] Diseases/names (n_features=59)
  > [group] Text (n_features=373)
  > [group] Drugs/Chem (n_features=419)
  > [group] Diseases (n_features=59)
  > [combo] Text+Drugs/Chem (n_features=792)
  > [combo] Text+Diseases (n_features=432)
  > [combo] Drugs/Chem+Diseases (n_features=478)
  > [combo] All (Text+Drugs/Chem+Diseases) (n_features=851)

=== Bootstrap CIs for ALL ablation levels – Phase II ===
Loading model: hint_xgb_model.joblib
  > [subgroup] Text/criteria (n_features=371)
  > [subgroup] Drugs/names (n_features=366)
  > [subgroup] Chem/SMILES (n_features=32)
  > [subgroup] Diseases/names (n_features=62)
  > [group] Text (n_features=371)
  > [group] Drugs/Chem (n_features=398)
  > [group] Diseases (n_features=62)
  > [combo] Text+Drugs/Chem (n_fea

,phase,group_level,label,delta_roc_auc_boot,delta_roc_auc_ci_low,delta_roc_auc_ci_high,delta_pr_auc_boot,delta_pr_auc_ci_low,delta_pr_auc_ci_high,delta_bal_acc_boot,delta_bal_acc_ci_low,delta_bal_acc_ci_high
0,I,subgroup,Text/criteria,0.100557,0.083025,0.117985,0.047305,0.036929,0.057056,0.071334,0.055268,0.088455
1,I,subgroup,Drugs/names,0.001420,-0.001320,0.004136,0.000312,-0.001953,0.002273,0.008097,0.000097,0.015513
2,I,subgroup,Chem/SMILES,0.000872,0.000129,0.001750,0.000414,-0.000061,0.000958,0.002988,-0.002677,0.009288
3,I,subgroup,Diseases/names,0.001080,-0.002040,0.004295,-0.000164,-0.001595,0.001179,0.004735,-0.007542,0.017941
4,I,group,Text,0.100557,0.083025,0.117985,0.047305,0.036929,0.057056,0.071334,0.055268,0.088455



Loaded all-phases CSV from: ablations\results\csv\ctod_all_phases.csv
Shape before merge: (33, 17)
Columns in all_df: ['phase', 'group_level', 'ablation_type', 'label', 'n_columns', 'roc_auc', 'pr_auc', 'bal_acc', 'delta_roc_auc', 'delta_pr_auc', 'delta_bal_acc', 'rel_roc_loss', 'rel_pr_loss', 'rel_balacc_loss', 'baseline_roc_auc', 'baseline_pr_auc', 'baseline_bal_acc']
Shape after merge: (33, 26)
[OK] Updated all-phases CSV with CI columns: ablations\results\csv\ctod_all_phases.csv
[OK] Updated Excel summary: ablations\results\excel\ablation_summary.xlsx
[OK] Saved CI plot: ablations\results\plots\ctod_phase_I_coarse_ci.png
[OK] Saved CI plot: ablations\results\plots\ctod_phase_II_coarse_ci.png
[OK] Saved CI plot: ablations\results\plots\ctod_phase_III_coarse_ci.png


In [25]:
# Build summary tables for PR-AUC and Balanced Accuracy
# including bootstrap Δ + 95% CIs
# ---------------------------------------------

# If not already defined in a previous cell, derive these from all_df
# (safe to overwrite even if they already exist)
coarse_df = all_df[all_df["group_level"] == "group"].copy()
subgroup_df = all_df[all_df["group_level"] == "subgroup"].copy()
combo_df   = all_df[all_df["group_level"] == "combo"].copy()

# ======== PR-AUC (point estimates + relative loss) ========
value_cols_pr = ["delta_pr_auc", "rel_pr_loss", "pr_auc"]

coarse_pivot_pr = make_pivot_table(coarse_df, value_cols=value_cols_pr)
print("\nCoarse groups – PR-AUC (point estimates):")
display(coarse_pivot_pr)

subgroup_pivot_pr = make_pivot_table(subgroup_df, value_cols=value_cols_pr)
print("\nSubgroups – PR-AUC (point estimates):")
display(subgroup_pivot_pr)

combo_pivot_pr = make_pivot_table(combo_df, value_cols=value_cols_pr)
print("\nCombination groups – PR-AUC (point estimates):")
display(combo_pivot_pr)


# ======== PR-AUC (bootstrap Δ + 95% CI) ========
# Assumes CI code created:
#   delta_pr_auc_boot, delta_pr_auc_ci_low, delta_pr_auc_ci_high

value_cols_pr_ci = [
    "delta_pr_auc_boot",
    "delta_pr_auc_ci_low",
    "delta_pr_auc_ci_high",
]

coarse_pivot_pr_ci = make_pivot_table(coarse_df, value_cols=value_cols_pr_ci)
print("\nCoarse groups – PR-AUC (bootstrap Δ + 95% CI):")
display(coarse_pivot_pr_ci)

subgroup_pivot_pr_ci = make_pivot_table(subgroup_df, value_cols=value_cols_pr_ci)
print("\nSubgroups – PR-AUC (bootstrap Δ + 95% CI):")
display(subgroup_pivot_pr_ci)

combo_pivot_pr_ci = make_pivot_table(combo_df, value_cols=value_cols_pr_ci)
print("\nCombination groups – PR-AUC (bootstrap Δ + 95% CI):")
display(combo_pivot_pr_ci)


# ======== Balanced Accuracy (point estimates + relative loss) ========
value_cols_bal = ["delta_bal_acc", "rel_balacc_loss", "bal_acc"]

coarse_pivot_bal = make_pivot_table(coarse_df, value_cols=value_cols_bal)
print("\nCoarse groups – Balanced Accuracy (point estimates):")
display(coarse_pivot_bal)

subgroup_pivot_bal = make_pivot_table(subgroup_df, value_cols=value_cols_bal)
print("\nSubgroups – Balanced Accuracy (point estimates):")
display(subgroup_pivot_bal)

combo_pivot_bal = make_pivot_table(combo_df, value_cols=value_cols_bal)
print("\nCombination groups – Balanced Accuracy (point estimates):")
display(combo_pivot_bal)


# ======== Balanced Accuracy (bootstrap Δ + 95% CI) ========
# Assumes CI code created:
#   delta_bal_acc_boot, delta_bal_acc_ci_low, delta_bal_acc_ci_high

value_cols_bal_ci = [
    "delta_bal_acc_boot",
    "delta_bal_acc_ci_low",
    "delta_bal_acc_ci_high",
]

coarse_pivot_bal_ci = make_pivot_table(coarse_df, value_cols=value_cols_bal_ci)
print("\nCoarse groups – Balanced Accuracy (bootstrap Δ + 95% CI):")
display(coarse_pivot_bal_ci)

subgroup_pivot_bal_ci = make_pivot_table(subgroup_df, value_cols=value_cols_bal_ci)
print("\nSubgroups – Balanced Accuracy (bootstrap Δ + 95% CI):")
display(subgroup_pivot_bal_ci)

combo_pivot_bal_ci = make_pivot_table(combo_df, value_cols=value_cols_bal_ci)
print("\nCombination groups – Balanced Accuracy (bootstrap Δ + 95% CI):")
display(combo_pivot_bal_ci)


# ---------------------------------------------
# Export to Excel (PR-AUC + BalAcc, with and without CIs)
# ---------------------------------------------
excel_dir = ROOT / "ablations" / "results" / "excel"
excel_dir.mkdir(parents=True, exist_ok=True)

output_path = excel_dir / "ablation_report_tables_PR_BalAcc.xlsx"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:

    # PR-AUC sheets (point estimates)
    coarse_pivot_pr.to_excel(writer, sheet_name="coarse_PR", index=False)
    subgroup_pivot_pr.to_excel(writer, sheet_name="subgroups_PR", index=False)
    combo_pivot_pr.to_excel(writer, sheet_name="combos_PR", index=False)

    # PR-AUC CI sheets
    coarse_pivot_pr_ci.to_excel(writer, sheet_name="coarse_PR_CI", index=False)
    subgroup_pivot_pr_ci.to_excel(writer, sheet_name="subgroups_PR_CI", index=False)
    combo_pivot_pr_ci.to_excel(writer, sheet_name="combos_PR_CI", index=False)

    # Balanced Accuracy sheets (point estimates)
    coarse_pivot_bal.to_excel(writer, sheet_name="coarse_BalAcc", index=False)
    subgroup_pivot_bal.to_excel(writer, sheet_name="subgroups_BalAcc", index=False)
    combo_pivot_bal.to_excel(writer, sheet_name="combos_BalAcc", index=False)

    # Balanced Accuracy CI sheets
    coarse_pivot_bal_ci.to_excel(writer, sheet_name="coarse_BalAcc_CI", index=False)
    subgroup_pivot_bal_ci.to_excel(writer, sheet_name="subgroups_BalAcc_CI", index=False)
    combo_pivot_bal_ci.to_excel(writer, sheet_name="combos_BalAcc_CI", index=False)

print("\n[OK] Exported PR-AUC & Balanced Accuracy tables (with CIs) to:", output_path)



Coarse groups – PR-AUC (point estimates):


,label,delta_pr_auc_I,delta_pr_auc_II,delta_pr_auc_III,pr_auc_I,pr_auc_II,pr_auc_III,rel_pr_loss_I,rel_pr_loss_II,rel_pr_loss_III
0,Diseases,-0.000149,-0.001431,0.003606,0.961505,0.917958,0.934163,-0.000155,-0.001561,0.003845
1,Drugs/Chem,0.000605,0.001511,-0.004403,0.960750,0.915016,0.942172,0.000629,0.001649,-0.004696
2,Text,0.046985,0.019065,0.014617,0.914370,0.897463,0.923152,0.048874,0.020801,0.015587



Subgroups – PR-AUC (point estimates):


,label,delta_pr_auc_I,delta_pr_auc_II,delta_pr_auc_III,pr_auc_I,pr_auc_II,pr_auc_III,rel_pr_loss_I,rel_pr_loss_II,rel_pr_loss_III
0,Chem/SMILES,0.000412,0.000409,-0.000444,0.960944,0.916119,0.938212,0.000428,0.000446,-0.000473
1,Diseases/names,-0.000149,-0.001431,0.003606,0.961505,0.917958,0.934163,-0.000155,-0.001561,0.003845
2,Drugs/names,0.000411,0.001117,-0.003459,0.960945,0.915411,0.941227,0.000427,0.001219,-0.003688
3,Text/criteria,0.046985,0.019065,0.014617,0.914370,0.897463,0.923152,0.048874,0.020801,0.015587



Combination groups – PR-AUC (point estimates):


...


Subgroups – Balanced Accuracy (bootstrap Δ + 95% CI):


,label,delta_bal_acc_boot_I,delta_bal_acc_boot_II,delta_bal_acc_boot_III,delta_bal_acc_ci_high_I,delta_bal_acc_ci_high_II,delta_bal_acc_ci_high_III,delta_bal_acc_ci_low_I,delta_bal_acc_ci_low_II,delta_bal_acc_ci_low_III
0,Chem/SMILES,0.002988,-0.001107,-0.000640,0.009288,0.002138,0.006097,-0.002677,-0.004108,-0.007757
1,Diseases/names,0.004735,-0.005504,0.001844,0.017941,-0.001249,0.010412,-0.007542,-0.010167,-0.006879
2,Drugs/names,0.008097,0.000680,0.026582,0.015513,0.006416,0.038686,0.000097,-0.004513,0.013883
3,Text/criteria,0.071334,0.043147,0.111489,0.088455,0.053052,0.129713,0.055268,0.033031,0.094921



Combination groups – Balanced Accuracy (bootstrap Δ + 95% CI):


,label,delta_bal_acc_boot_I,delta_bal_acc_boot_II,delta_bal_acc_boot_III,delta_bal_acc_ci_high_I,delta_bal_acc_ci_high_II,delta_bal_acc_ci_high_III,delta_bal_acc_ci_low_I,delta_bal_acc_ci_low_II,delta_bal_acc_ci_low_III
0,All (Text+Drugs/Chem+Diseases),0.073731,0.002416,0.113941,0.091419,0.009877,0.131889,0.057174,-0.005443,0.096417
1,Drugs/Chem+Diseases,0.010143,-0.006191,0.029306,0.024355,-0.000184,0.041623,-0.002536,-0.012222,0.015115
2,Text+Diseases,0.071206,0.026614,0.122401,0.088474,0.035710,0.139671,0.054070,0.017231,0.105306
3,Text+Drugs/Chem,0.063969,0.011476,0.098448,0.080608,0.019813,0.117292,0.048182,0.003957,0.081277



[OK] Exported PR-AUC & Balanced Accuracy tables (with CIs) to: C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\ablations\results\excel\ablation_report_tables_PR_BalAcc.xlsx


In [27]:
import pandas as pd
from pathlib import Path

# If ROOT is not defined, set it
try:
    ROOT
except NameError:
    ROOT = Path(".")

# ----------------------------------------------------
# Helper to build a pretty table for one metric
# ----------------------------------------------------
def make_metric_table(df_phase_level, metric: str):
    if metric == "roc":
        delta_boot, ci_low, ci_high = (
            "delta_roc_auc_boot",
            "delta_roc_auc_ci_low",
            "delta_roc_auc_ci_high",
        )
        rel_loss, score = "rel_roc_loss", "roc_auc"
        delta_name, score_name = "ΔROC-AUC (boot)", "ROC-AUC after ablation"

    elif metric == "pr":
        delta_boot, ci_low, ci_high = (
            "delta_pr_auc_boot",
            "delta_pr_auc_ci_low",
            "delta_pr_auc_ci_high",
        )
        rel_loss, score = "rel_pr_loss", "pr_auc"
        delta_name, score_name = "ΔPR-AUC (boot)", "PR-AUC after ablation"

    elif metric == "bal":
        delta_boot, ci_low, ci_high = (
            "delta_bal_acc_boot",
            "delta_bal_acc_ci_low",
            "delta_bal_acc_ci_high",
        )
        rel_loss, score = "rel_balacc_loss", "bal_acc"
        delta_name, score_name = "ΔBalanced Accuracy (boot)", "Balanced Accuracy after ablation"

    else:
        raise ValueError("metric must be 'roc', 'pr', or 'bal'")

    out = df_phase_level[
        ["phase", "label", delta_boot, ci_low, ci_high, rel_loss, score]
    ].copy()

    out.rename(columns={
        "phase": "Phase",
        "label": "Group",
        delta_boot: delta_name,
        ci_low: "CI low",
        ci_high: "CI high",
        rel_loss: "Relative loss",
        score: score_name,
    }, inplace=True)

    out.sort_values(["Phase", "Group"], inplace=True)
    return out


# ----------------------------------------------------
# Build all tables (phase × level × metric)
# ----------------------------------------------------
levels = {"group": "group", "subgroup": "subgroup", "combo": "combo"}
metrics = ["roc", "pr", "bal"]
phases = sorted(all_df["phase"].unique())

tables = {}

for phase in phases:
    df_phase = all_df[all_df["phase"] == phase]
    for level_name, level_value in levels.items():
        df_level = df_phase[df_phase["group_level"] == level_value]
        for metric in metrics:
            sheet = f"{metric.upper()}_{phase}_{level_name}"
            tables[sheet] = make_metric_table(df_level, metric)

# ----------------------------------------------------
# Write to Excel 
# ----------------------------------------------------
excel_dir = ROOT / "ablations" / "results"
excel_dir.mkdir(parents=True, exist_ok=True)
output_path = excel_dir / "ablation_PR_BA_Tables_per_phase.xlsx"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    for sheet_name, df in tables.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

    workbook = writer.book

    header_fmt = workbook.add_format({
        "font_name": "Times New Roman",
        "font_size": 11,
        "bold": True,
        "border": 1,
        "align": "center",
        "valign": "vcenter",
    })

    body_fmt = workbook.add_format({
        "font_name": "Times New Roman",
        "font_size": 11,
        "border": 1,
        "align": "center",
        "valign": "vcenter",
    })

    for sheet_name, df in tables.items():
        ws = writer.sheets[sheet_name]
        nrows, ncols = df.shape

        # Write header with border
        for col in range(ncols):
            ws.write(0, col, df.columns[col], header_fmt)

        # Write body with borders
        for row in range(1, nrows + 1):
            for col in range(ncols):
                ws.write(row, col, df.iloc[row - 1, col], body_fmt)

        # Adjust column width slightly
        ws.set_column(0, ncols - 1, 22)

print("\n[OK] Excel file with fully bordered tables (Times New Roman 11) saved to:")
print(output_path)



[OK] Excel file with fully bordered tables (Times New Roman 11) saved to:
C:\Users\Maria\OneDrive\Ambiente de Trabalho\ablation study (updated with feedabck)\ablations\results\ablation_PR_BA_Tables_per_phase.xlsx
